In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-status-prediction/loan_data.csv


Loading the data set

In [2]:


df = pd.read_csv("/kaggle/input/loan-status-prediction/loan_data.csv")

df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [3]:
df = df.drop(['Loan_ID'],axis = 1)

In [4]:
df.shape

(381, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             376 non-null    object 
 1   Married            381 non-null    object 
 2   Dependents         373 non-null    object 
 3   Education          381 non-null    object 
 4   Self_Employed      360 non-null    object 
 5   ApplicantIncome    381 non-null    int64  
 6   CoapplicantIncome  381 non-null    float64
 7   LoanAmount         381 non-null    float64
 8   Loan_Amount_Term   370 non-null    float64
 9   Credit_History     351 non-null    float64
 10  Property_Area      381 non-null    object 
 11  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 35.8+ KB


In [6]:
df.isnull().sum()

Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
df = df.dropna(subset=['Gender','Dependents','Self_Employed','Loan_Amount_Term','Credit_History'])

In [8]:
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:
df['Property_Area'].mode()


0    Semiurban
Name: Property_Area, dtype: object

converting categorical values to numerical values

In [10]:
def cat_to_num(df, c_var):
    for i in c_var:
        uniques_value = df[i].unique()
        df[i].replace(uniques_value, [0, 1], inplace=True)

    for i in ['Property_Area']:
        uniques_value = df[i].unique()
        df[i].replace(uniques_value, [0, 1, 3], inplace=True)
    for i in ['Dependents']:
        unique_values = df[i].unique()
        df[i].replace(unique_values, [0,1,2,3],inplace = True)

In [11]:
c_variables = ['Gender', 'Married', 'Education','Self_Employed', 'Loan_Status']
cat_to_num(df,c_variables)

/tmp/ipykernel_19/3915073759.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].replace(uniques_value, [0, 1], inplace=True)
/tmp/ipykernel_19/3915073759.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[i].replace(uniques_value, [0, 1], inplace=True)
/tmp/ipykernel_19/3915073759.py:8: F

In [12]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,0,0,0,0,4583,1508.0,128.0,360.0,1.0,0,0
1,0,0,1,0,1,3000,0.0,66.0,360.0,1.0,1,1
2,0,0,1,1,0,2583,2358.0,120.0,360.0,1.0,1,1
3,0,1,1,0,0,6000,0.0,141.0,360.0,1.0,1,1
4,0,0,1,1,0,2333,1516.0,95.0,360.0,1.0,1,1


processing the data into X, y

In [13]:
X = df.drop(['Loan_Status'],axis = 1)
y = df['Loan_Status']

In [14]:
X.shape,y.shape

((308, 11), (308,))

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308 entries, 0 to 380
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             308 non-null    int64  
 1   Married            308 non-null    int64  
 2   Dependents         308 non-null    int64  
 3   Education          308 non-null    int64  
 4   Self_Employed      308 non-null    int64  
 5   ApplicantIncome    308 non-null    int64  
 6   CoapplicantIncome  308 non-null    float64
 7   LoanAmount         308 non-null    float64
 8   Loan_Amount_Term   308 non-null    float64
 9   Credit_History     308 non-null    float64
 10  Property_Area      308 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 28.9 KB


Splitting data set into train and test

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)


In [17]:
X_train.shape, X_test.shape

((246, 11), (62, 11))

standardise the data (x-mu/s)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Model decision tree classifier

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [20]:
Model = DecisionTreeClassifier(max_depth = 3,min_samples_leaf = 32)

In [21]:
Model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=32)

Testing part

In [22]:
y_pred = Model.predict(X_test)

In [23]:
accuracy = accuracy_score(y_test,y_pred)
print(f"accuracy score: {accuracy}")

accuracy score: 0.8064516129032258
